In [1]:
import torch
import pandas as pd
import numpy as np
#import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold,StratifiedKFold 
import gc
import os
from os import path
from sys import path as systemPath
systemPath.append(path.join('..', '..'))
from scipy.spatial import distance_matrix
#import argparse
import random
import matplotlib.pyplot as plt
import networkx as nx
import MDAnalysis as mda
from matplotlib.ticker import MaxNLocator,MultipleLocator
from MDAnalysis.core.groups import AtomGroup
import seaborn as sns
import statistics as st

In [ ]:
# real position index
mutation_pos = [29, 29, 35, 35, 35, 41, 41, 50, 59, 62, 77, 86, 89, 92, 92, 95, 101, 101, 104, 107, 107, 110, 113, 113, 119, 119, 122, 122, 137, 137, 143, 143, 149, 149, 152, 161, 164, 164, 170, 170, 176, 176, 188, 191, 191, 194, 203, 203, 206, 209, 215, 218, 218, 221, 221, 227, 227, 227, 230, 233, 236, 236, 236, 239, 239, 242, 242, 245, 245, 248, 251, 260, 260, 263, 266, 269, 272, 293, 299, 302, 302, 314, 320, 329, 341, 365, 365, 368, 368, 371, 371, 392, 398, 398, 398, 404, 407, 425, 425, 431, 431, 431, 455, 464, 464, 467, 473, 476, 488, 494, 497, 524, 527, 539, 539, 542, 542, 557, 557, 566, 575, 575, 581, 581, 593, 596, 602, 605, 608, 611, 614, 617, 626, 626, 647, 653, 653, 659, 659, 662, 671, 674, 677, 680, 680, 683, 689, 692, 695, 698, 704, 704, 707, 713, 716, 725, 734, 755, 764, 782, 785, 788, 800, 812, 818, 839, 848, 860, 860, 860, 863, 866, 878, 884, 884, 887, 890, 896, 899, 899, 902, 914, 914, 917, 920, 923, 926, 929, 938, 944, 962, 980, 983, 983, 989, 989, 992, 1004, 1007, 1007, 1010, 1013, 1013, 1016, 1019, 1022, 1025, 116, 1028, 125, 131, 158, 164, 32, 32, 32, 179, 182, 191, 200, 35, 206, 206, 212, 218, 227, 38, 242, 41, 275, 275, 293, 308, 308, 329, 347, 50, 365, 374, 53, 386, 407, 410, 56, 416, 419, 431, 440, 443, 59, 59, 59, 452, 470, 473, 485, 506, 521, 524, 551, 560, 569, 572, 578, 581, 587, 596, 599, 602, 608, 662, 662, 665, 671, 671, 677, 686, 689, 692, 692, 701, 704, 86, 716, 719, 722, 734, 770, 812, 812, 815, 854, 101, 878, 893, 917, 932, 947, 971, 113, 995, 1091, 1097, 1103, 1106, 1139, 1163, 1166, 1166, 1175, 1175, 1184, 1190, 1220, 1226, 1229, 1229, 1238, 1247, 1247, 1253, 1259, 1259, 1262, 1262, 1268, 1271, 1280, 1283, 1289, 1295, 1298, 1301, 1301, 1301, 1304, 1304, 1307, 1310, 1310, 1310, 1310, 1313, 1313, 1319, 1319, 1322, 1322, 1331, 1331, 1337, 1340, 1343, 1343, 1349, 1349, 1358, 1361, 1364, 1364, 1379, 1382, 1391, 1391, 1397, 1400, 1406, 1418, 1421, 1427, 1433, 1442, 1451, 1451, 1457, 1457, 1463, 1466, 1472, 1481, 1484, 1484, 1484, 1490, 1493, 1499, 1499, 1502, 1505, 1514, 1517, 1517, 1520, 1526, 1535, 1559, 1565, 1568, 1568, 1571, 1574, 1574, 1589, 1601, 1607, 1610, 1613, 1619, 1634, 1634, 1637, 1643, 1649, 1649, 1649, 1655, 1673, 1673, 1685, 1685, 1688, 1691, 1694, 1703, 1709, 1712, 1718, 1724, 1730, 1736, 1739, 1739, 1742, 1745, 1751, 1757, 1757, 1760, 1763, 1766, 1769, 1769, 1778, 1802, 1808, 1820, 1826, 1832, 1850, 1853, 1865, 1868, 1874, 1877, 1889, 1910, 1919, 1922, 1928, 1928, 1943, 1946, 1946, 1955, 1961, 1970, 1979, 1985, 1988, 2000, 2000, 2009, 2012, 2015, 2024, 2030, 2039, 2051, 2054, 2057, 2063, 2069, 2072, 2075, 2075, 2075, 2075, 1163, 2063, 2063, 2066, 1166, 1169, 1172, 1172, 1172, 1184, 1190, 1220, 1232, 1238, 1241, 1241, 1244, 1250, 1256, 1262, 1262, 1277, 1277, 1085, 1286, 1289, 1292, 1310, 1088, 1313, 1325, 1328, 1358, 1370, 1094, 1373, 1415, 1424, 1427, 1100, 1454, 1490, 1523, 1535, 1538, 1541, 1544, 1112, 1598, 1607, 1619, 1634, 1634, 1646, 1673, 1682, 1688, 1706, 1712, 1718, 1721, 1727, 1733, 1736, 1742, 1745, 1760, 1763, 1769, 1775, 1775, 1778, 1784, 1787, 1790, 1793, 1796, 1814, 1820, 1826, 1829, 1847, 1142, 1853, 1907, 1916, 1916, 1928, 1934, 1955, 1955, 1958, 1958, 1964, 1967, 1976, 1997, 2003, 2018, 2057]
# mutation type
mutation_name = ["ALA" , "ASP", "CYS", "SER", "ASP", "ARG", "VAL", "ARG", "CYS", "CYS", "ASP", "ARG", "CYS", "ARG", "GLU", "ARG", "ARG", "VAL", "GLU", "SER", "VAL", "CYS", "ARG", "ASP", "SER", "ASP", "SER", "ALA", "ARG", "ASP", "SER", "ARG", "ARG", "ASP", "ARG", "GLU", "ARG", "ASP", "ARG", "ALA", "CYS", "SER", "ARG", "CYS", "SER", "CYS", "VAL", "ALA", "SER", "SER", "ALA", "CYS", "VAL", "CYS", "SER", "ARG", "CYS", "SER", "GLU", "SER", "CYS", "ALA", "ASP", "CYS", "ASP", "CYS", "SER", "SER", "CYS", "SER", "ALA", "CYS", "ALA", "SER", "CYS", "VAL", "VAL", "CYS", "SER", "ARG", "GLU", "ARG", "ALA", "CYS", "CYS", "CYS", "ALA", "CYS", "SER", "CYS", "ASP", "ALA", "ARG", "SER", "CYS", "ARG", "ARG", "SER", "ALA", "CYS", "SER", "ASP", "ARG", "CYS", "SER", "SER", "ARG", "VAL", "ARG", "SER", "ALA", "VAL", "SER", "CYS", "SER", "CYS", "SER", "SER", "ASP", "ARG", "CYS", "ASP", "SER", "VAL", "SER", "CYS", "VAL", "SER", "CYS", "SER", "SER", "SER", "CYS", "SER", "SER", "VAL", "ALA", "SER", "ARG", "CYS", "ALA", "ALA", "SER", "ARG", "ASP", "ARG", "ASP", "SER", "CYS", "ALA", "CYS", "SER", "CYS", "SER", "SER", "SER", "CYS", "GLU", "CYS", "CYS", "CYS", "ARG", "ASP", "ARG", "VAL", "CYS", "SER", "SER", "ALA", "VAL", "ARG", "SER", "SER", "SER", "CYS", "SER", "SER", "SER", "SER", "ASP", "ALA", "SER", "CYS", "SER", "CYS", "SER", "ALA", "SER", "ASP", "ALA", "CYS", "SER", "CYS", "ASP", "SER", "VAL", "ARG", "CYS", "SER", "CYS", "ASP", "CYS", "SER", "VAL", "SER", "VAL", "VAL", "GLU", "ASP", "SER", "VAL", "VAL", "SER", "ARG", "SER", "ALA", "CYS", "ASP", "ASP", "SER", "ARG", "VAL", "ASP", "ALA", "SER", "ALA", "VAL", "ALA", "SER", "CYS", "ASP", "ASP", "VAL", "ALA", "SER", "VAL", "GLU", "ASP", "ARG", "ARG", "ALA", "ASP", "ARG", "ASP", "SER", "ARG", "VAL", "ARG", "ASP", "ARG", "ASP", "SER", "ARG", "SER", "SER", "SER", "SER", "SER", "ALA", "SER", "SER", "VAL", "VAL", "ARG", "ASP", "SER", "SER", "ASP", "SER", "SER", "ARG", "ALA", "ASP", "ASP", "VAL", "ASP", "ARG", "VAL", "ARG", "VAL", "VAL", "ALA", "GLU", "ASP", "VAL", "SER", "ASP", "ASP", "ASP", "ALA", "ASP", "ARG", "GLU", "ARG", "ASP", "ARG", "SER", "SER", "ALA", "SER", "ASP", "ASP", "ASP", "ASP", "SER", "VAL", "VAL", "SER", "ASP", "ARG", "ASP", "ASP", "VAL", "ASP", "ASP", "ASP", "VAL", "ASP", "ASP", "VAL", "SER", "ALA", "VAL", "ALA", "VAL", "VAL", "ARG", "ASP", "ARG", "ARG", "SER", "GLU", "SER", "CYS", "ASP", "ASP", "VAL", "SER", "CYS", "SER", "ARG", "PRO", "ARG", "SER", "CYS", "SER", "SER", "CYS", "CYS", "SER", "SER", "TRP", "SER", "ALA", "ASP", "VAL", "GLU", "VAL", "CYS", "SER", "SER", "VAL", "SER", "ASP", "SER", "SER", "GLU", "ARG", "GLU", "CYS", "SER", "ALA", "ASP", "VAL", "VAL", "GLU", "GLU", "GLU", "VAL", "ASP", "SER", "ASP", "VAL", "ARG", "SER", "ARG", "GLU", "CYS", "GLU", "CYS", "SER", "VAL", "ARG", "ASP", "CYS", "ARG", "SER", "CYS", "SER", "VAL", "SER", "ASP", "SER", "ARG", "VAL", "ASP", "GLU", "VAL", "SER", "VAL", "ASP", "ASP", "ARG", "VAL", "ASP", "ASP", "CYS", "ASP", "SER", "ASP", "ASP", "SER", "SER", "CYS", "CYS", "ARG", "GLU", "SER", "VAL", "VAL", "SER", "VAL", "VAL", "SER", "SER", "ARG", "SER", "CYS", "ASP", "ARG", "SER", "ALA", "CYS", "SER", "SER", "ARG", "VAL", "SER", "CYS", "SER", "ASP", "ASP", "SER", "ARG", "ARG", "CYS", "ARG", "SER", "SER", "CYS", "ASP", "VAL", "ASP", "VAL", "VAL", "ASP", "SER", "SER", "ARG", "ARG", "GLU", "ARG", "ASP", "SER", "SER", "CYS", "ASP", "ASP", "VAL", "VAL", "CYS", "ALA", "ARG", "ARG", "SER", "ASP", "VAL", "GLU", "ASP", "VAL", "ASP", "ARG", "ASP", "ASP", "VAL", "CYS", "VAL", "GLU", "ARG", "SER", "ARG", "ARG", "GLU", "ARG", "ARG", "ARG", "ASP", "SER", "CYS", "SER", "VAL", "CYS", "SER", "ARG", "ASP", "ARG", "ASP", "ASP", "GLU", "ARG", "GLU", "ASP", "ARG", "ARG", "SER", "ALA", "ASP", "SER", "ASP", "SER", "VAL", "SER", "ASP", "ARG", "VAL", "ASP", "ASP", "CYS", "ASP", "CYS", "ASP", "ARG", "VAL", "SER", "CYS", "GLU", "ARG", "VAL", "CYS", "GLU", "ASP", "CYS", "VAL", "ASP", "ALA", "CYS", "SER", "VAL", "ARG", "ALA", "GLU", "VAL", "SER", "CYS", "SER", "SER", "ASP", "VAL", "ASP", "GLU", "ASP", "ASP", "GLU", "ARG", "ASP", "ASP", "ASP", "SER", "ARG", "SER", "VAL", "ASP", "ASP", "ARG", "ASP", "ASP", "CYS"]
# lethal:0, non-lethal:1 
lethal = [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1]  
# triple number 
triplet_number = [5, 5, 7, 7, 7, 9, 9, 12, 15, 16, 21, 24, 25, 26, 26, 27, 29, 29, 30, 31, 31, 32, 33, 33, 35, 35, 36, 36, 41, 41, 43, 43, 45, 45, 46, 49, 50, 50, 52, 52, 54, 54, 58, 59, 59, 60, 63, 63, 64, 65, 67, 68, 68, 69, 69, 71, 71, 71, 72, 73, 74, 74, 74, 75, 75, 76, 76, 77, 77, 78, 79, 82, 82, 83, 84, 85, 86, 93, 95, 96, 96, 100, 102, 105, 109, 117, 117, 118, 118, 119, 119, 126, 128, 128, 128, 130, 131, 137, 137, 139, 139, 139, 147, 150, 150, 151, 153, 154, 158, 160, 161, 170, 171, 175, 175, 176, 176, 181, 181, 184, 187, 187, 189, 189, 193, 194, 196, 197, 198, 199, 200, 201, 204, 204, 211, 213, 213, 215, 215, 216, 219, 220, 221, 222, 222, 223, 225, 226, 227, 228, 230, 230, 231, 233, 234, 237, 240, 247, 250, 256, 257, 258, 262, 266, 268, 275, 278, 282, 282, 282, 283, 284, 288, 290, 290, 291, 292, 294, 295, 295, 296, 300, 300, 301, 302, 303, 304, 305, 308, 310, 316, 322, 323, 323, 325, 325, 326, 330, 331, 331, 332, 333, 333, 334, 335, 336, 337, 34, 338, 37, 39, 48, 50, 6, 6, 6, 55, 56, 59, 62, 7, 64, 64, 66, 68, 71, 8, 76, 9, 87, 87, 93, 98, 98, 105, 111, 12, 117, 120, 13, 124, 131, 132, 14, 134, 135, 139, 142, 143, 15, 15, 15, 146, 152, 153, 157, 164, 169, 170, 179, 182, 185, 186, 188, 189, 191, 194, 195, 196, 198, 216, 216, 217, 219, 219, 221, 224, 225, 226, 226, 229, 230, 24, 234, 235, 236, 240, 252, 266, 266, 267, 280, 29, 288, 293, 301, 306, 311, 319, 33, 327, 10, 12, 14, 15, 26, 34, 35, 35, 38, 38, 41, 43, 53, 55, 56, 56, 59, 62, 62, 64, 66, 66, 67, 67, 69, 70, 73, 74, 76, 78, 79, 80, 80, 80, 81, 81, 82, 83, 83, 83, 83, 84, 84, 86, 86, 87, 87, 90, 90, 92, 93, 94, 94, 96, 96, 99, 100, 101, 101, 106, 107, 110, 110, 112, 113, 115, 119, 120, 122, 124, 127, 130, 130, 132, 132, 134, 135, 137, 140, 141, 141, 141, 143, 144, 146, 146, 147, 148, 151, 152, 152, 153, 155, 158, 166, 168, 169, 169, 170, 171, 171, 176, 180, 182, 183, 184, 186, 191, 191, 192, 194, 196, 196, 196, 198, 204, 204, 208, 208, 209, 210, 211, 214, 216, 217, 219, 221, 223, 225, 226, 226, 227, 228, 230, 232, 232, 233, 234, 235, 236, 236, 239, 247, 249, 253, 255, 257, 263, 264, 268, 269, 271, 272, 276, 283, 286, 287, 289, 289, 294, 295, 295, 298, 300, 303, 306, 308, 309, 313, 313, 316, 317, 318, 321, 323, 326, 330, 331, 332, 334, 336, 337, 338, 338, 338, 338, 34, 334, 334, 335, 35, 36, 37, 37, 37, 41, 43, 53, 57, 59, 60, 60, 61, 63, 65, 67, 67, 72, 72, 8, 75, 76, 77, 83, 9, 84, 88, 89, 99, 103, 11, 104, 118, 121, 122, 13, 131, 143, 154, 158, 159, 160, 161, 17, 179, 182, 186, 191, 191, 195, 204, 207, 209, 215, 217, 219, 220, 222, 224, 225, 227, 228, 233, 234, 236, 238, 238, 239, 241, 242, 243, 244, 245, 251, 253, 255, 256, 262, 27, 264, 282, 285, 285, 289, 291, 298, 298, 299, 299, 301, 302, 305, 312, 314, 319, 332]
# chain 
alpha12 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [4]:
##儲存資料
radius_all=np.load(f'D:/collagen/new_uh_rad/traj_radius_all.npy', allow_pickle = True)
radius_wt=np.load(f'D:/collagen/new_uh_rad/traj_radius_wt.npy', allow_pickle = True)
unit_hei_all=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei.npy', allow_pickle = True)
unit_hei_wt=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei_wt.npy', allow_pickle = True)

gpo_radius_traj = np.load(f'D:/collagen/gpo30/ana/gpo_traj_radius.npy', allow_pickle = True)
gpo_unit_hei_traj = np.load(f'D:/collagen/gpo30/ana/gpo_traj_unhei.npy', allow_pickle = True)

mutation_name_arr = np.load(f'D:/collagen/data/mutation_name.npy')  # (587,)
lethal_arr = np.load(f'D:/collagen/data/lethal.npy')  # (587,)
alpha12_arr = np.load(f'D:/collagen/data/alpha12.npy')  # (587,)
triplet_number_arr = np.load(f'D:/collagen/data/triplet_number.npy')  # (587,)

In [ ]:
##AMINO ACID ANA
# ---------- 1. 先準備 offset 與四大類對照 ----------
OFFSETS = [0,
           -1,-2,-4,-5,-7,-8,-10,-11,
            1, 2, 4, 5, 7, 8, 10, 11]

AA_CLASS = {
    'Small':       {"GLY","ALA","CYS","SER","THR"},
    'Charged':     {"HIS","LYS","ARG","GLU","ASP"},
    'Hydrophobic': {"VAL","ILE","LEU","MET","PHE","TYR","TRP"},
    'Polar':       {"ASN","GLN"},
}

def aa_to_class(aa):
    """把三字母 mapping 到四大類；沒對到就回傳 'Other'"""
    for cls, aset in AA_CLASS.items():
        if aa in aset:
            return cls
    return 'Other'


# ---------- 2. 主迴圈：走過每一筆突變 ----------
aa17_all = []          # 每筆突變一列，存 17 個 AA 種類
for tri_pos, mpos, mtype, lethal, a12 in zip(triplet_number,
                                             mutation_pos,
                                             mutation_name,
                                             lethal_arr,
                                             alpha12_arr):

    # 2-1  讀檔 → 建 MDAnalysis Universe
    file_psf = (f"D:/collagen/a{a12}/{tri_pos}_{mtype.upper()}_{a12}/"
                f"mutation_{tri_pos}_{mtype.upper()}__{lethal}_{a12}.psf")
    file_pdb = f"D:/collagen/a{a12}_dcd_cut/{tri_pos}_{mtype.upper()}.dcd"
    u = mda.Universe(file_psf, file_pdb)

    # 2-2  判斷突變在哪一條鏈，並記 resid0（該鏈 M0 的 resid）
    if   0    <= mpos <= 1054:
        seg_mut   = "0A"
        resid0    = 3*(tri_pos-1) + 17          # 公式 x1
    elif 1055 <= mpos <= 2080:
        seg_mut   = "0B"
        resid0    = 3*(tri_pos-1) + 1054 + 10   # 公式 x2
    else:
        seg_mut   = "0C"
        resid0    = 3*(tri_pos-1) + 17 + 2080   # 公式 x3

    # 2-3  把 17 個位置的 AA 名稱存進 aa_row
    aa_row = []
    for off in OFFSETS:
        resid = resid0 + off
        sel   = u.select_atoms(f"segid {seg_mut} and resid {resid}")
        aa3   = sel.resnames[0] if len(sel) else "NA"
        aa_row.append(aa_to_class(aa3))     # 轉成四大類名

    aa17_all.append(aa_row)

# ---------- 3. 轉成 numpy / pandas 後續統計 ----------
aa17_all = np.array(aa17_all)    # (n_mut, 17)
# 接下來可用 pandas.crosstab 計數，再畫九個位置 × 四顏色的長條圖


c:\Users\wendy\anaconda3\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [5]:
import os
import numpy as np

# ── 3. 轉成 numpy 陣列 ─────────────────────────
aa17_all = np.array(aa17_all)    # (n_mut, 17)

# ── 4. 指定輸出資料夾與檔名 ─────────────────────
out_dir  = "D:/collagen/amino_acid_type"
os.makedirs(out_dir, exist_ok=True)

out_path = os.path.join(out_dir, "aa17_class.npy")

# ── 5. 存檔 ────────────────────────────────────
np.save(out_path, aa17_all)
print(f"Saved → {out_path}")


NameError: name 'aa17_all' is not defined

In [4]:
# 計算每一種突變最後100個frame每個frame每個位置的radius
radius_all = []
for tri_pos, mpos, mtype, l, a12 in zip(triplet_number, mutation_pos, mutation_name, lethal, alpha12):
    # 以 tri_pos 當作 center（也可以改用 mutation_pos）
    center = tri_pos  
    # 建立 mutation ID（方便除錯與後續標籤）
    mutation_id = f"{tri_pos}_{mtype.upper()}_{a12}"
    
    # 依據公式計算各條鏈的基準殘基編號：
    # 原本：x1 = 3*(center-1)+17
    x1 = 3 * (center - 1) + 17
    x2 = 3 * (center - 1) + 1054 + 10
    x3 = 3 * (center - 1) + 17 + 2080
    
    # 計算前4後4共9個位置（原始程式用 range(-12,13,3)）
    # 這9個數值對應於相對位置 P4, P3, P2, P1, M0, P1', P2', P3', P4'
    x1_values = [x1 + i for i in range(-12, 13, 3)]
    x2_values = [x2 + i for i in range(-12, 13, 3)]
    x3_values = [x3 + i for i in range(-12, 13, 3)]
    
    # ----------------------------
    # mutant 結構計算
    # ----------------------------
    
    file_psf = f"D:/collagen/a{a12}/{tri_pos}_{mtype.upper()}_{a12}/mutation_{tri_pos}_{mtype.upper()}__{l}_{a12}.psf"
    file_pdb = f"D:/collagen/a{a12}_dcd_cut/{tri_pos}_{mtype.upper()}.dcd"
    u1 = mda.Universe(file_psf, file_pdb)

    radius_traj = [[] for _ in range(100)]
    for t in u1.trajectory:
        if a12 == 1:
            alpha1_1_mu = u1.select_atoms(f"segid 0A and resid {x1_values[4]}")  # 中心位置 index 4
            alpha1_2_mu = u1.select_atoms(f"segid 0C and resid {x3_values[4]}")
            alpha2_mu   = u1.select_atoms(f"segid 0B and resid {x2_values[4]}")
            if alpha1_1_mu.resnames[0] != mtype.upper() or alpha1_2_mu.resnames[0] != mtype.upper():
                print(f"Warning: Mutation {mutation_id} chain 0A or 0C not matching {mtype.upper()}")
            if alpha2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0B is not GLY")
        elif a12 == 2:
            alpha2_mu   = u1.select_atoms(f"segid 0B and resid {x2_values[4]}")
            alpha1_1_mu = u1.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u1.select_atoms(f"segid 0C and resid {x3_values[4]}")
            if alpha2_mu.resnames[0] != mtype.upper():
                print(f"Warning: Mutation {mutation_id} chain 0B not matching {mtype.upper()}")
            if alpha1_1_mu.resnames[0] != "GLY" or alpha1_2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0A or 0C is not GLY")
        
        # 計算 mutant 的半徑 profile：對9個位置逐一計算
        radius_profile_mut = []  # mutant 的半徑 profile
        for r1, r2, r3 in zip(x1_values, x2_values, x3_values):
            # 分別選取 mutant 結構中各鏈該位置的原子
            a1 = u1.select_atoms(f"segid 0A and resid {r1}")
            a2 = u1.select_atoms(f"segid 0B and resid {r2}")
            a3 = u1.select_atoms(f"segid 0C and resid {r3}")
            # 分別計算各原子群的幾何中心
            c1 = a1.center_of_geometry()
            c2 = a2.center_of_geometry()
            c3 = a3.center_of_geometry()
            # 合併三個原子群，計算整體幾何中心
            combined = AtomGroup(a1 + a2 + a3)
            combined_center = combined.center_of_geometry()
            # 計算各鏈中心與合併中心的距離，取平均作為此位置的半徑
            d1 = np.linalg.norm(c1 - combined_center)
            d2 = np.linalg.norm(c2 - combined_center)
            d3 = np.linalg.norm(c3 - combined_center)
            avg_dist = np.mean([d1, d2, d3])
            radius_profile_mut.append(avg_dist)
        # print(radius_profile_mut)
        radius_traj [t.frame] = radius_profile_mut
    radius_all.append(radius_traj)
radius_all = np.array (radius_all)

In [5]:
np.save(f'D:/collagen/new_uh_rad/traj_radius_all', radius_all)

In [6]:
# print(sum(radius_all[0,:,0])/100)
radius_avg = []
radius_std = []
for i in range (radius_all.shape[0]):
    pos_avg = []
    pos_std = []
    for j in range(radius_all.shape[2]):
        avg = st.mean(radius_all[i,:,j])
        std = st.stdev(radius_all[i,:,j])
        pos_avg.append(avg)
        pos_std.append(std)
    # print(pos_avg)
    radius_avg.append(pos_avg)
    radius_std.append(pos_std)
radius_avg = np.array (radius_avg)
radius_std = np.array (radius_std)

In [7]:
# 計算每一種突變最後100個frame每個frame每個位置的radius
radius_wt = []
for tri_pos, mpos, mtype, l, a12 in zip(triplet_number, mutation_pos, mutation_name, lethal, alpha12):
    # 以 tri_pos 當作 center（也可以改用 mutation_pos）
    center = tri_pos  
    # 建立 mutation ID（方便除錯與後續標籤）
    mutation_id = f"{tri_pos}_{mtype.upper()}_{a12}"
    
    # 依據公式計算各條鏈的基準殘基編號：
    # 原本：x1 = 3*(center-1)+17
    x1 = 3 * (center - 1) + 17
    x2 = 3 * (center - 1) + 1054 + 10
    x3 = 3 * (center - 1) + 17 + 2080
    
    # 計算前4後4共9個位置（原始程式用 range(-12,13,3)）
    # 這9個數值對應於相對位置 P4, P3, P2, P1, M0, P1', P2', P3', P4'
    x1_values = [x1 + i for i in range(-12, 13, 3)]
    x2_values = [x2 + i for i in range(-12, 13, 3)]
    x3_values = [x3 + i for i in range(-12, 13, 3)]
    
    # ----------------------------
    # mutant 結構計算
    # ----------------------------
    
    file_psf = f"D:/collagen/collagen_wt/collagen_wt.psf"
    file_pdb = f"D:/collagen/collagen_wt/collagen_wt_nvt.dcd"
    u_wt = mda.Universe(file_psf, file_pdb)

    radius_traj_wt = [[] for _ in range(100)]
    for t in u_wt.trajectory:
        if a12 == 1:
            alpha1_1_mu = u_wt.select_atoms(f"segid 0A and resid {x1_values[4]}")  # 中心位置 index 4
            alpha1_2_mu = u_wt.select_atoms(f"segid 0C and resid {x3_values[4]}")
            alpha2_mu   = u_wt.select_atoms(f"segid 0B and resid {x2_values[4]}")
            if alpha1_1_mu.resnames[0] != "GLY" or alpha1_2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0A or 0C not GLY")
            if alpha2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0B is not GLY")
        elif a12 == 2:
            alpha2_mu   = u_wt.select_atoms(f"segid 0B and resid {x2_values[4]}")
            alpha1_1_mu = u_wt.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u_wt.select_atoms(f"segid 0C and resid {x3_values[4]}")
            if alpha2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0B not GLY")
            if alpha1_1_mu.resnames[0] != "GLY" or alpha1_2_mu.resnames[0] != "GLY":
                print(f"Warning: Mutation {mutation_id} chain 0A or 0C is not GLY")

        radius_profile_wt = []  # WT 的半徑 profile
        for r1, r2, r3 in zip(x1_values, x2_values, x3_values):
            a1_wt = u_wt.select_atoms(f"segid 0A and resid {r1}")
            a2_wt = u_wt.select_atoms(f"segid 0B and resid {r2}")
            a3_wt = u_wt.select_atoms(f"segid 0C and resid {r3}")
            c1_wt = a1_wt.center_of_geometry()
            c2_wt = a2_wt.center_of_geometry()
            c3_wt = a3_wt.center_of_geometry()
            combined_wt = AtomGroup(a1_wt + a2_wt + a3_wt)
            combined_center_wt = combined_wt.center_of_geometry()
            d1_wt = np.linalg.norm(c1_wt - combined_center_wt)
            d2_wt = np.linalg.norm(c2_wt - combined_center_wt)
            d3_wt = np.linalg.norm(c3_wt - combined_center_wt)
            avg_dist_wt = np.mean([d1_wt, d2_wt, d3_wt])
            radius_profile_wt.append(avg_dist_wt)

            # print(radius_profile_mut)
            radius_traj_wt [t.frame] = radius_profile_wt
    radius_wt.append(radius_traj_wt)
radius_wt = np.array (radius_wt)

In [8]:
np.save(f'D:/collagen/new_uh_rad/traj_radius_wt', radius_wt)

In [9]:
# print(sum(radius_wt[0,:,0])/100)
radius_wt_avg = []
radius_wt_std = []
for i in range (radius_wt.shape[0]):
    pos_avg = []
    pos_std = []
    for j in range(radius_wt.shape[2]):
        avg = st.mean(radius_wt[i,:,j])
        std = st.stdev(radius_wt[i,:,j])
        pos_avg.append(avg)
        pos_std.append(std)
    # print(pos_avg)
    radius_wt_avg.append(pos_avg)
    radius_wt_std.append(pos_std)
radius_wt_avg = np.array (radius_wt_avg)
radius_wt_std = np.array (radius_wt_std)

In [40]:
print(radius_wt_avg)

[[6.43412532 6.74042346 8.04085984 ... 4.28344008 4.01697875 4.05167428]
 [6.43412532 6.74042346 8.04085984 ... 4.28344008 4.01697875 4.05167428]
 [8.04085984 4.98805279 3.52264151 ... 4.05167428 5.95997416 6.08059499]
 ...
 [5.90897658 4.74290631 4.55575871 ... 4.42406409 4.83230784 3.81512297]
 [3.8839824  4.42406409 4.83230784 ... 4.88361374 3.79770912 3.57014903]
 [4.72045536 5.10080341 4.68297647 ... 3.13634853 4.17901227 7.16017121]]


In [10]:
# 計算每一種突變最後100個frame每個frame每個位置的unit_hei
unit_hei_all = []
for tri_pos, mpos, mtype, l, a12 in zip(triplet_number, mutation_pos, mutation_name, lethal, alpha12):
    # 以 tri_pos 當作 center（也可以改用 mutation_pos）
    center = tri_pos  
    # 建立 mutation ID（方便除錯與後續標籤）
    mutation_id = f"{tri_pos}_{mtype.upper()}_{a12}"
    
    # 依據公式計算各條鏈的基準殘基編號：
    # 原本：x1 = 3*(center-1)+17
    x1 = 3 * (center - 1) + 17
    x2 = 3 * (center - 1) + 1054 + 10
    x3 = 3 * (center - 1) + 17 + 2080
    
    # 計算前4後4共9個位置（原始程式用 range(-12,13,3)）
    # 這9個數值對應於相對位置 P4, P3, P2, P1, M0, P1', P2', P3', P4'
    x1_values = [x1 + i for i in range(-12, 16, 3)]
    x2_values = [x2 + i for i in range(-12, 16, 3)]
    x3_values = [x3 + i for i in range(-12, 16, 3)]
    
    # ----------------------------
    # mutant 結構計算
    # ----------------------------
    
    file_psf = f"D:/collagen/a{a12}/{tri_pos}_{mtype.upper()}_{a12}/mutation_{tri_pos}_{mtype.upper()}__{l}_{a12}.psf"
    file_pdb = f"D:/collagen/a{a12}_dcd_cut/{tri_pos}_{mtype.upper()}.dcd"
    u1 = mda.Universe(file_psf, file_pdb)

    unit_hei_traj = [[] for _ in range(100)]
    for t in u1.trajectory:
        if a12 == 1:
            alpha1_1_mu = u1.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u1.select_atoms(f"segid 0C and resid {x3_values[4]}")
            alpha2_mu = u1.select_atoms(f"segid 0B and resid {x2_values[4]}")
            if len(alpha1_1_mu) == 0 or len(alpha1_2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0A/0C")
            else:
                if alpha1_1_mu.resnames[0] != mtype.upper() or alpha1_2_mu.resnames[0] != mtype.upper():
                    print(f"Warning: Mutation {mutation_id} chain 0A or 0C not matching {mtype.upper()}")
            if len(alpha2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0B")
            else:
                if alpha2_mu.resnames[0] != "GLY":
                    print(f"Warning: Mutation {mutation_id} chain 0B is not GLY")
        elif a12 == 2:
            alpha2_mu = u1.select_atoms(f"segid 0B and resid {x2_values[4]}")
            alpha1_1_mu = u1.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u1.select_atoms(f"segid 0C and resid {x3_values[4]}")
            if len(alpha2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0B")
            else:
                if alpha2_mu.resnames[0] != mtype.upper():
                    print(f"Warning: Mutation {mutation_id} chain 0B not matching {mtype.upper()}")
            if len(alpha1_1_mu) == 0 or len(alpha1_2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0A/0C")
            else:
                if alpha1_1_mu.resnames[0] != "GLY" or alpha1_2_mu.resnames[0] != "GLY":
                    print(f"Warning: Mutation {mutation_id} chain 0A or 0C is not GLY")
        
        # 計算 mutant 的單位長 profile：
        # 先計算每個位置的組合幾何中心，存入列表 unit_mass_center_mu（共有 9 個點）
        unit_mass_center_mu = []
        for r1, r2, r3 in zip(x1_values, x2_values, x3_values):
            a1 = u1.select_atoms(f"segid 0A and resid {r1}")
            a2 = u1.select_atoms(f"segid 0B and resid {r2}")
            a3 = u1.select_atoms(f"segid 0C and resid {r3}")
            # 分別計算三個原子群的幾何中心
            c1 = a1.center_of_geometry()
            c2 = a2.center_of_geometry()
            c3 = a3.center_of_geometry()
            # 合併三個原子群後計算總中心
            combined = AtomGroup(a1 + a2 + a3)
            combined_center = combined.center_of_geometry()
            unit_mass_center_mu.append(combined_center)
        # 接著計算連續兩點之間的距離（共 8 個距離值，即單位長 profile）
        unit_hei_profile_mut = []
        for i in range(len(unit_mass_center_mu) - 1):
            distance_mu = np.linalg.norm(unit_mass_center_mu[i] - unit_mass_center_mu[i+1])
            unit_hei_profile_mut.append(distance_mu)
        # print(unit_hei_profile_mut)
        unit_hei_traj [t.frame] = unit_hei_profile_mut
    unit_hei_all.append(unit_hei_traj)
unit_hei_all = np.array (unit_hei_all)

In [11]:
np.save(f'D:/collagen/new_uh_rad/traj_unit_hei', unit_hei_all)

In [12]:
# 計算每一種突變最後100個frame每個frame每個位置的unit_hei
unit_hei_wt = []
for tri_pos, mpos, mtype, l, a12 in zip(triplet_number, mutation_pos, mutation_name, lethal, alpha12):
    # 以 tri_pos 當作 center（也可以改用 mutation_pos）
    center = tri_pos  
    # 建立 mutation ID（方便除錯與後續標籤）
    mutation_id = f"{tri_pos}_{mtype.upper()}_{a12}"
    
    # 依據公式計算各條鏈的基準殘基編號：
    # 原本：x1 = 3*(center-1)+17
    x1 = 3 * (center - 1) + 17
    x2 = 3 * (center - 1) + 1054 + 10
    x3 = 3 * (center - 1) + 17 + 2080
    
    # 計算前4後4共9個位置（原始程式用 range(-12,13,3)）
    # 這9個數值對應於相對位置 P4, P3, P2, P1, M0, P1', P2', P3', P4'
    x1_values = [x1 + i for i in range(-12, 16, 3)]
    x2_values = [x2 + i for i in range(-12, 16, 3)]
    x3_values = [x3 + i for i in range(-12, 16, 3)]
    
    # ----------------------------
    # mutant 結構計算
    # ----------------------------
    
    file_psf = f"D:/collagen/collagen_wt/collagen_wt.psf"
    file_pdb = f"D:/collagen/collagen_wt/collagen_wt_nvt.dcd"
    u_wt = mda.Universe(file_psf, file_pdb)

    unit_hei_traj_wt = [[] for _ in range(100)]
    for t in u_wt.trajectory:
        if a12 == 1:
            alpha1_1_mu = u_wt.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u_wt.select_atoms(f"segid 0C and resid {x3_values[4]}")
            alpha2_mu = u_wt.select_atoms(f"segid 0B and resid {x2_values[4]}")
            if len(alpha1_1_mu) == 0 or len(alpha1_2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0A/0C")
            else:
                if alpha1_1_mu.resnames[0] != mtype.upper() or alpha1_2_mu.resnames[0] != mtype.upper():
                    print(f"Warning: Mutation {mutation_id} chain 0A or 0C not matching {mtype.upper()}")
            if len(alpha2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0B")
            else:
                if alpha2_mu.resnames[0] != "GLY":
                    print(f"Warning: Mutation {mutation_id} chain 0B is not GLY")
        elif a12 == 2:
            alpha2_mu = u_wt.select_atoms(f"segid 0B and resid {x2_values[4]}")
            alpha1_1_mu = u_wt.select_atoms(f"segid 0A and resid {x1_values[4]}")
            alpha1_2_mu = u_wt.select_atoms(f"segid 0C and resid {x3_values[4]}")
            if len(alpha2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0B")
            else:
                if alpha2_mu.resnames[0] != mtype.upper():
                    print(f"Warning: Mutation {mutation_id} chain 0B not matching {mtype.upper()}")
            if len(alpha1_1_mu) == 0 or len(alpha1_2_mu) == 0:
                print(f"Warning: Mutation {mutation_id} missing atoms in chain 0A/0C")
            else:
                if alpha1_1_mu.resnames[0] != "GLY" or alpha1_2_mu.resnames[0] != "GLY":
                    print(f"Warning: Mutation {mutation_id} chain 0A or 0C is not GLY")
        
        # 計算 mutant 的單位長 profile：
        # 先計算每個位置的組合幾何中心，存入列表 unit_mass_center_mu（共有 9 個點）
        unit_mass_center_wt = []
        for r1, r2, r3 in zip(x1_values, x2_values, x3_values):
            a1_wt = u_wt.select_atoms(f"segid 0A and resid {r1}")
            a2_wt = u_wt.select_atoms(f"segid 0B and resid {r2}")
            a3_wt = u_wt.select_atoms(f"segid 0C and resid {r3}")
            combined_wt = AtomGroup(a1_wt + a2_wt + a3_wt)
            combined_center_wt = combined_wt.center_of_geometry()
            unit_mass_center_wt.append(combined_center_wt)
        unit_hei_profile_wt = []
        for i in range(len(unit_mass_center_wt) - 1):
            distance_wt = np.linalg.norm(unit_mass_center_wt[i] - unit_mass_center_wt[i+1])
            unit_hei_profile_wt.append(distance_wt)
            # print(unit_hei_profile_mut)
            unit_hei_traj_wt [t.frame] = unit_hei_profile_wt
    unit_hei_wt.append(unit_hei_traj_wt)
unit_hei_wt = np.array (unit_hei_wt)

In [13]:
np.save(f'D:/collagen/new_uh_rad/traj_unit_hei_wt', unit_hei_wt)

In [ ]:
radius_all=np.load(f'D:/collagen/new_uh_rad/traj_radius.npy', allow_pickle = True)
radius_wt=np.load(f'D:/collagen/new_uh_rad/traj_radius_wt.npy', allow_pickle = True)
unit_hei_all=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei.npy', allow_pickle = True)
unit_hei_wt=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei_wt.npy', allow_pickle = True)
print(radius_all.shape)
print(radius_wt.shape)
print(unit_hei_all.shape)
print(unit_hei_wt.shape)